In [1]:
#                                                   LIBRARIES
import os
import sys
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# OPEN API ETABS DOCUMENTATION 
ruta = r"C:\Program Files\Computers and Structures\ETABS 20\CSI API ETABS v1.chm"
os.startfile(ruta)

In [4]:
#                                               INIT INSTANCE
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)

# myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject")     # Abrir una nueva ventana de Etabs
myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject")            # Trabajar con una ventana abierta de Etabs

# myETABSObject.ApplicationStart()

SapModel = myETABSObject.SapModel               # Crear un objeto SapModel

SapModel.InitializeNewModel()                   # Iniciar Modelo

SapModel.File.NewBlank()                        # Crear un modelo en blanco

0

In [5]:
#                                                   INSERT UNITS
kgf_m_C = 8
SapModel.SetPresentUnits(kgf_m_C)

MATERIAL_CONCRETE = 2
SapModel.PropMaterial.SetMaterial('CONCRETO280', MATERIAL_CONCRETE)
E = 15000*(280**0.5)*10000   # (kgf/m2)
SapModel.PropMaterial.SetMPIsotropic('CONCRETO280', E, 0.2, 9.90E-06)
SapModel.PropMaterial.SetWeightAndMass('CONCRETO280', 1, 2400)
SapModel.PropMaterial.SetOConcrete('CONCRETO280', 2800000, False, 0, 1, 2, 0.0022, 0.0052)

MATERIAL_REBAR = 6
SapModel.PropMaterial.SetMaterial('REBAR', MATERIAL_REBAR)
SapModel.PropMaterial.SetMPIsotropic('REBAR', 2.0E10, 0.2, 9.90E-06)
SapModel.PropMaterial.SetWeightAndMass('REBAR', 1, 7850)
SapModel.PropMaterial.SetORebar("REBAR", 42000000, 63000000, 46000000, 69000000, 2, 2, 0.02, 0.1, False)

0

In [6]:
#                                                   DEFINE: FRAMES
# FrameObj Interface
# Definir las propiedades de Seccion a un Frame rectangular
SapModel.PropFrame.SetRectangle('R1',               # Nombre del nuevo frame
                                'CONCRETO280',      # Nombre del Material, debe existir
                                0.6,                # peralte
                                0.3 )               # base
                                
# Datos extra para Frame Vigas        
SapModel.PropFrame.SetRebarBeam("R1",               # Nombre de un frame existente
                                "REBAR",            # Material del acero longitudinal
                                "REBAR",            # Material del acero transversal
                                0.06,               # Recubrimiento superior   6cm
                                0.06,               # Recubrimiento inferior   6cm
                                1.29*10**-4,        # Top Left Area
                                1.29*10**-4,        # Top Right Area
                                1.29*10**-4,        # Bottom Left Area
                                1.29*10**-4)        # Bottom Right Area

## Añadir objeto Frame por Coordenadas
#FrameName1 = ' '
#[FrameName1, _] = SapModel.FrameObj.AddByCoord(0, 0, 0,      # Coordenadas X,Y,Z del punto inicial
#                                              0, 0, 10,      # Coordenadas X,Y,Z del punto final
#                                              FrameName1,    # Vacio, el programa asignará un nombre por defecto
#                                              'R1')          # Nombre de la seccion, debe existir el nombre de esa seccion, opcional
#FrameName2 = ' '
#FrameName3 = ' '
#[FrameName2, ret] = SapModel.FrameObj.AddByCoord(0, 0, 10, 8, 0, 16, FrameName2, 'R1')
#[FrameName3, ret] = SapModel.FrameObj.AddByCoord(-4, 0, 10, 0, 0, 10, FrameName3, 'R1')

0

In [7]:
# MODEL FRAMES CON RESTRAINTS
b = 7   # m
B = 7   # m
H = 2.7 # m
n_vanos = 4  # Número de vanos

# Crear una lista para almacenar los nombres de los marcos
FrameNames = []

# Crear columnas y vigas para cada vano
for i in range(n_vanos - 1):
    # Nombres de los marcos para el vano actual
    col_name1 = f'Column_{i}_1'
    col_name2 = f'Column_{i}_2'
    beam_name1 = f'Beam_{i}_1'
    beam_name2 = f'Beam_{i}_2'

    # Columnas
    [col1, ret] = SapModel.FrameObj.AddByCoord(i * B, 0, 0, i * B, 0, H, col_name1, 'R1')
    [col2, ret] = SapModel.FrameObj.AddByCoord(i * B + b, 0, 0, i * B + b, 0, H, col_name2, 'R1')
    
    # Vigas
    [beam1, ret] = SapModel.FrameObj.AddByCoord(i * B, 0, H, (i + 1) * B, 0, H, beam_name1, 'R1')
    [beam2, ret] = SapModel.FrameObj.AddByCoord(i * B + b, 0, H, (i + 1) * B + b, 0, H, beam_name2, 'R1')

    # Agregar los nombres de los marcos a la lista
    FrameNames.extend([col_name1, col_name2, beam_name1, beam_name2])

# Agregar la última columna y su viga correspondiente
last_col_name1 = f'Column_{n_vanos - 1}_1'
last_col_name2 = f'Column_{n_vanos - 1}_2'
[last_col1, ret] = SapModel.FrameObj.AddByCoord((n_vanos - 1) * B, 0, 0, (n_vanos - 1) * B, 0, H, last_col_name1, 'R1')
[last_col2, ret] = SapModel.FrameObj.AddByCoord((n_vanos - 1) * B + b, 0, 0, (n_vanos - 1) * B + b, 0, H, last_col_name2, 'R1')
FrameNames.extend([last_col_name1, last_col_name2])

# Imprimir los nombres de los marcos creados
print("Nombres de los marcos creados:")
print(FrameNames)

# Aplicar restricciones a los nodos creados
Restraint = [True,      # U1
             True,      # U2
             True,      # U3
             True,      # R1
             False,     # R2
             False]     # R3

for frame_name in FrameNames:
    # Obtener el nombre del nodo asociado al marco
    node_name = frame_name + '.1'
    # Aplicar restricciones al nodo
    SapModel.PointObj.SetRestraint(node_name, Restraint)

# Mensaje de confirmación
print("Restricciones aplicadas a los nodos creados.")

# Lista para almacenar los nombres de los nodos
NodeNames = []

# Obtener los nombres de los nodos asociados a los marcos creados
for frame_name in FrameNames:
    # Obtener el nombre del nodo asociado al marco
    node_name = frame_name + '.1'
    # Agregar el nombre del nodo a la lista
    NodeNames.append(node_name)

# Imprimir los nombres de los nodos
print("Nombres de los nodos asociados a los marcos:")
print(NodeNames)



Nombres de los marcos creados:
['Column_0_1', 'Column_0_2', 'Beam_0_1', 'Beam_0_2', 'Column_1_1', 'Column_1_2', 'Beam_1_1', 'Beam_1_2', 'Column_2_1', 'Column_2_2', 'Beam_2_1', 'Beam_2_2', 'Column_3_1', 'Column_3_2']
Restricciones aplicadas a los nodos creados.
Nombres de los nodos asociados a los marcos:
['Column_0_1.1', 'Column_0_2.1', 'Beam_0_1.1', 'Beam_0_2.1', 'Column_1_1.1', 'Column_1_2.1', 'Beam_1_1.1', 'Beam_1_2.1', 'Column_2_1.1', 'Column_2_2.1', 'Beam_2_1.1', 'Beam_2_2.1', 'Column_3_1.1', 'Column_3_2.1']


In [8]:
#                                                       DEFINE: SLABS
SapModel.PropArea.SetSlab("Losa Macisa",          # Nombre de la Losa
                          0,                      # Slab Type - Slab
                          1,                      # ShellType, ShellThin
                          "CONCRETO280",          # Material
                          0.15)                     # Thickness  

#SapModel.PropArea.SetSlab("Slab Ribbed2", 3, 1, "CONCRETO280", 0.20)
#SapModel.PropArea.SetSlabRibbed("Slab Ribbed2", 0.20, 0.05, 0.1, 0.1, 0.4, 1)
# # Definir las propiedades de la Losa Slab
# SapModel.PropArea.SetSlab("Slab Ribbed",          # Nombre de la Losa
#                           3,                      # Slab Type - Ribbed
#                           3,                      # ShellType, ShellThin
#                           "CONCRETO280",          # Material
#                           0.15)                   # Thickness  

# # Definir las propiedades de la Losa Ribbed
# SapModel.PropArea.SetSlabRibbed("Slab Ribbed",   # Nombre de la Losa
#                                  11.1,           # OverallDepth              
#                                  22.2,           # SlabThickness
#                                  33.3,           # StemWidthTop
#                                  44.4,           # StemWidthBottom
#                                  55.5,           # RibSpacing
#                                  2)              # RibsParallelTo - Local Axis

0

In [9]:
# MODEL: SLABS

L = 7   # m
b = 7   # m
B = 7   # m
H = 2.7 # m
AreaName = ' '
x = [0,B,b,0]
y = [0,0,L,L]
z = [H,H,H,H]
SapModel.AreaObj.AddByCoord(4, # Numero de puntos que definen el area
                            x, # Coordenadas X
                            y, # Coordenadas Y
                            z, # Coordenadas Z
                            AreaName, # Vacio, el programa asignará un nombre por defecto
                            "Losa Macisa")  # Nombre de la seccion, debe existir el nombre de esa seccion, opcional

SapModel.View.RefreshView(0,          # 0: Refresh todas las ventanas
                        False)        # False: No matener el zoom

0

In [ ]:
#                                               DEFINE: WALLS
SapModel.PropArea.SetWall("Placa",            # Name
                          1,                  # WallPropType , Seccion Especificada por el usuario
                          1,                  # Shell Type - Shell Thin   
                          "CONCRETO280",      # Nombre del Material
                          0.20)               # Thickness

# MyValue = [1, 1, 1, 1, 1, 1, 1, 1, 1]
# SapModel.PropArea.SetModifiers("Placa", MyValue)


0

In [ ]:
# MODEL: WALLS

AreaName = ' '
x = [0,2,2,0]
y = [0,0,0,0]
z = [0,0,3,3]
SapModel.AreaObj.AddByCoord(4, # Numero de puntos que definen el area
                            x, # Coordenadas X
                            y, # Coordenadas Y
                            z, # Coordenadas Z
                            AreaName, # Vacio, el programa asignará un nombre por defecto
                            "Placa")  # Nombre de la seccion, debe existir el nombre de esa seccion, opcional
SapModel.View.RefreshView(0,          # 0: Refresh todas las ventanas
                        False)        # False: No matener el zoom#

0

In [10]:
# Fuerza lateral de 500 kN
lateral_force = 500  # kN

# Iterar sobre cada nodo asociado a los marcos del pórtico
for node_name in NodeNames:
    # Insertar la fuerza lateral en el nodo en dirección del eje x (global)
    ret = SapModel.PointObj.SetLoadForce(node_name, 'GLOBAL', [lateral_force, 0, 0])
    
    # Verificar si la inserción de la carga fue exitosa
    if ret:
        print(f"Fuerza lateral de {lateral_force} kN insertada en el nodo {node_name} en dirección del eje x (GLOBAL).")
    else:
        print(f"Error al insertar la fuerza lateral en el nodo {node_name} en dirección del eje x (GLOBAL).")

## Añadir carga lateral al pórtico
#
## Nombre del nodo donde se aplicará la fuerza lateral (por ejemplo, el primer nodo de la primera columna)
#node_name = 'FrameName1.1'
#
## Fuerza lateral de 100 kN
#lateral_force = 300  # kN
#
## Insertar la fuerza lateral en el nodo
#ret = SapModel.PointObj.SetLoadForce(node_name, 'GLOBAL', [lateral_force, 0, 0])
#
## Verificar si la inserción de la carga fue exitosa
#if ret:
#    print("Fuerza lateral insertada correctamente en el nodo:", node_name)
#else:
#    print("Error al insertar la fuerza lateral en el nodo:", node_name)

Fuerza lateral de 500 kN insertada en el nodo Column_0_1.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Column_0_2.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Beam_0_1.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Beam_0_2.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Column_1_1.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Column_1_2.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Beam_1_1.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Beam_1_2.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Column_2_1.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Column_2_2.1 en dirección del eje x (GLOBAL).
Fuerza lateral de 500 kN insertada en el nodo Beam_2_1.1 en dirección del eje x (GLOBAL)

In [12]:
# SAVE MODEL
ModelPath = r"D:\Softwares\Etabs v20\ejemplo_1"
SapModel.File.Save(ModelPath)

0

In [13]:
# RUN ANALYSIS
SapModel.Analyze.RunAnalysis()

0

In [ ]:
# GET: TABLES OF ANALYSIS

# Obtener la tabla "Joint Reactions"
name_table = "Joint Reactions"
tables = SapModel.DatabaseTables.GetTableForDisplayArray(name_table, GroupName="")

# Verificar el tipo y contenido de tables
print("Tipo de tables:", type(tables))
print("Contenido de tables:", tables)

# Si tables es una tupla, asumimos que el primer elemento es la tabla que queremos
if isinstance(tables, tuple):
    table = tables[0]
    print("Tabla obtenida correctamente.")
else:
    print("Error al obtener la tabla.")

# Nombre del archivo Excel de salida
excel_file = "Joint_Reactions.xlsx"

# Ruta completa del archivo Excel
excel_path = os.path.join(r"D:\Softwares\Etabs v20\JointReactions", excel_file)

# Exportar la tabla a un archivo Excel si table es un objeto válido
if hasattr(table, "ExportSpreadsheet"):
    ret = table.ExportSpreadsheet(excel_path)
    # Verificar si la exportación fue exitosa
    if ret:
        print(f"La tabla '{name_table}' se ha exportado correctamente a '{excel_file}'.")
    else:
        print(f"Error al exportar la tabla '{name_table}' a '{excel_file}'.")
else:
    print("No se puede exportar la tabla: table no es un objeto válido.")


Tipo de tables: <class 'list'>
Contenido de tables: [(), 2, ('Story', 'Label', 'UniqueName', 'OutputCase', 'CaseType', 'StepType', 'StepNumber', 'StepLabel', 'FX', 'FY', 'FZ', 'MX', 'MY', 'MZ'), 16, ('Base', '1', '1', 'Dead', 'LinStatic', None, None, None, '1440.57', '1002.17', '5707.86', '0', '0', '0', 'Base', '1', '1', 'Live', 'LinStatic', None, None, None, '0', '0', '0', '0', '0', '0', 'Base', '2', '2', 'Dead', 'LinStatic', None, None, None, '-466.79', '564.56', '7233.85', '0', '0', '0', 'Base', '2', '2', 'Live', 'LinStatic', None, None, None, '0', '0', '0', '0', '0', '0', 'Base', '6', '6', 'Dead', 'LinStatic', None, None, None, '-375.83', '802.45', '8201.99', '0', '0', '0', 'Base', '6', '6', 'Live', 'LinStatic', None, None, None, '0', '0', '0', '0', '0', '0', 'Base', '9', '9', 'Dead', 'LinStatic', None, None, None, '1.53', '60.89', '3829.79', '0', '0', '0', 'Base', '9', '9', 'Live', 'LinStatic', None, None, None, '0', '0', '0', '0', '0', '0', 'Base', '11', '11', 'Dead', 'LinStatic'